# **PINECONE VECTOR DATABASE :**

In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pinecone/Pinecone.pdf


## Import libraries : 

In [30]:
!pip uninstall -y pinecone-client pinecone
!pip install -U pinecone langchain-pinecone
!pip install -U langchain-community  # also needed for vectorstores

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
from langchain_pinecone import Pinecone


In [32]:
!pip install langchain-groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
!pip install langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import langchain
import pinecone
from langchain_pinecone import Pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

In [35]:
def doc_loader(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [36]:
pdf=doc_loader("/kaggle/input/pinecone")

In [37]:
len(pdf)

16

In [38]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=50
)

In [39]:
documents=text_splitter.split_documents(pdf)

In [40]:
len(documents)

35

# ChatGroq Model : 

In [41]:
import os 
os.environ["GROQ_API_KEY"]="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"

In [42]:
llm=ChatGroq(model="gemma2-9b-it")

## HuggingFace Embeddings : 

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings

In [44]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [45]:
vectors=embeddings.embed_query("How are you?")
#vectors

## Pinecone vector Database : 

In [46]:
from pinecone import Pinecone

In [47]:
os.environ["PINECONE_API_KEY"]="pcsk_3nmAVN_RmEN1VznseKbztWfPji5BX2WkppkrnnyKQZB8tg2wySHbDCMCzejoXSY9v2WtxC"
environment="us-east1-gcp"

In [48]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"],environment=environment)

In [49]:
print("Existing indexes:", pc.list_indexes().names())

Existing indexes: ['langchainvector']


## Creating Index : 

In [50]:
index_name="langchainvector"

In [51]:
!pip install --upgrade "pinecone-client>=2.2.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)


### Checking if index present or not :

In [52]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print(f"Deleted old index: {index_name}")

Deleted old index: langchainvector


In [53]:
from pinecone import ServerlessSpec

In [54]:

pc.create_index(
    name=index_name,
    dimension=384,  # must match your embeddings
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
print(f"Created new index: {index_name} with dimension 384")


Created new index: langchainvector with dimension 384


In [55]:
from langchain_pinecone import PineconeVectorStore

In [56]:
vectorstore = PineconeVectorStore.from_documents(
    documents=documents,  # your split documents
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=os.environ["PINECONE_API_KEY"]
)


# Cosine Similarity Retreive Results from VectorDB :

In [59]:
def retrieve_query(query,k=2):
    matching_results=vectorstore.similarity_search(query,k=k)
    return matching_results

# Question Answering Using Langchain : 

In [60]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq

In [61]:
llm=ChatGroq(model="gemma2-9b-it")

In [62]:
chain=load_qa_chain(llm,chain_type="stuff")

/tmp/ipykernel_36/1983183188.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain=load_qa_chain(llm,chain_type="stuff")


## Search answers from VectorDB :

In [63]:
def retrieve_answer(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [65]:
our_query="What is Pinecone?"
answer=retrieve_answer(our_query)
print(answer)

[Document(id='ada60b72-3156-448d-8971-dfa6e380f232', metadata={'creationdate': '2024-08-15T11:25:51-07:00', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'moddate': '2024-08-15T11:25:53-07:00', 'page': 1.0, 'page_label': '2', 'producer': 'Adobe PDF Library 17.0', 'source': '/kaggle/input/pinecone/Pinecone.pdf', 'total_pages': 16.0, 'trapped': '/False'}, page_content='Search for meaning everywhere with Pinecone and AWS .....................................16'), Document(id='beb19ad5-6d82-4efa-b0d1-edc12cd7baba', metadata={'creationdate': '2024-08-15T11:25:51-07:00', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'moddate': '2024-08-15T11:25:53-07:00', 'page': 9.0, 'page_label': '10', 'producer': 'Adobe PDF Library 17.0', 'source': '/kaggle/input/pinecone/Pinecone.pdf', 'total_pages': 16.0, 'trapped': '/False'}, page_content='10\nImage-to-image search\nIn traditional image search engines, you typically use text queries to find images, and the search engine returns results \nbased on keywo